In [ ]:
%run "../../../common/0_notebooks_base_setup.py"

! pip install uk-covid19

<img src='../../../common/logo_DH.png' align='left' width=35%/>

# Checkpoint APIs

---

En esta práctica vamos a usar una api que disponibiliza datos de COVID-19 en Reino Unido

La documentación de la API está disponible en https://coronavirus.data.gov.uk/details/developers-guide

Todos los pedidos (requests) a la API son sobre HTTPS.

También proveen una biblioteca para acceso a los datos https://github.com/publichealthengland/coronavirus-dashboard-api-python-sdk

En la primera parte de esta práctica vamos a usar un request para consultar cierta información (<a href="https://coronavirus.data.gov.uk/details/developers-guide#sdks">documentación</a>), y en la segunda parte vamos a consultar la misma info usando la bilbioteca que provee <a href="https://github.com/publichealthengland/coronavirus-dashboard-api-python-sdk">Public Health England</a>


## Imports


In [ ]:
import pandas as pd
import numpy as np
from requests import get
from json import dumps
from datetime import date, timedelta

from uk_covid19 import Cov19API

## Ejercicio 1

Usando un web request construir un DataFrame con información sobre casos nuevos y muertes en Inglaterra ("england") el día de ayer:

Los valores de metricName que queremos obtener en la respuesta son:

* date,
* areaName,
* areaCode,
* newCasesByPublishDate,
* cumCasesByPublishDate,
* newDeaths28DaysByPublishDate,
* cumDeaths28DaysByPublishDate

Para ver qué filtros podemos aplicar en la consulta:

https://coronavirus.data.gov.uk/details/developers-guide#params-filters

Según la documentación, la estructura de la respuesta queda definida por 

`structure={[responseName]:[metricName], [responseName]:[metricName]}`

Para ver los valores disponibles para metricName:

https://coronavirus.data.gov.uk/details/developers-guide `See a list of valid metrics for structure`

**Vemos en la documentación que el filtro areaType es requerido para todas las consultas:**

`The areaType metric is mandatory and must be defined in all queries.`





In [ ]:
hoy = date.today()
ayer = hoy + timedelta(days=-1)
print(ayer)
print(str(ayer))


In [ ]:
ENDPOINT = "https://api.coronavirus.data.gov.uk/v1/data"

#valores para los filtros:

AREA_TYPE = "nation"
AREA_NAME = "england"
DATE = str(ayer)

filters = [
    f"areaType={ AREA_TYPE }",
    f"areaName={ AREA_NAME }",
    f"date={ DATE }"
]

# estructura de la respuesta

structure = {
    "date": "date",
    "name": "areaName",
    "code": "areaCode",
    "dailyCases": "newCasesByPublishDate",
    "cumulativeCases": "cumCasesByPublishDate",
    "dailyDeaths": "newDeaths28DaysByPublishDate",
    "cumulativeDeaths": "cumDeaths28DaysByPublishDate"
}

api_params = {
    "filters": str.join(";", filters),
    "structure": dumps(structure, separators=(",", ":")),
    "format":"json"
}


response = get(ENDPOINT, params = api_params, timeout=10)

if response.status_code >= 400:
    raise RuntimeError(f'Request failed: { response.text }')

print(response.url)
print("---")
print(response.json())


Veamos qué datos vienen en el response

In [ ]:
response_dict = response.json()
response_dict.keys()

In [ ]:
# cantidad de registros

response_dict['length']

In [ ]:
response_dict['maxPageLimit']

In [ ]:
response_dict['data']

In [ ]:
# qué llega al server en el request:

response_dict['requestPayload']

In [ ]:
response_dict['pagination']

In [ ]:
response_df = pd.DataFrame(response.json()["data"])
response_df

In [ ]:
response.json()["data"]

# Ejercicio 2

Obtener todos los datos que obtuvimos en el ejercicio anterior para los últimos 30 días.


Quitamos el valor de `date` de los filtros y vemos que la primera página trae en orden decreciente de fecha más de treinta días

In [ ]:
ENDPOINT = "https://api.coronavirus.data.gov.uk/v1/data"

#valores para los filtros:

AREA_TYPE = "nation"
AREA_NAME = "england"

filters = [
    f"areaType={ AREA_TYPE }",
    f"areaName={ AREA_NAME }"
]

# estructura de la respuesta

structure = {
    "date": "date",
    "name": "areaName",
    "code": "areaCode",
    "dailyCases": "newCasesByPublishDate",
    "cumulativeCases": "cumCasesByPublishDate",
    "dailyDeaths": "newDeaths28DaysByPublishDate",
    "cumulativeDeaths": "cumDeaths28DaysByPublishDate"
}

api_params = {
    "filters": str.join(";", filters),
    "structure": dumps(structure, separators=(",", ":")),
    "format": "json",
    "page": "1"
}


response = get(ENDPOINT, params = api_params, timeout=10)

if response.status_code >= 400:
    raise RuntimeError(f'Request failed: { response.text }')

print(response.url)
print("---")
#print(response.json())

In [ ]:
response_df = pd.DataFrame(response.json()["data"])
response_df

Filtramos por las fechas de interés

In [ ]:
max_date = date.today()
min_date = max_date + timedelta(days=-30)

mask = np.logical_and(pd.to_datetime(response_df.date) <= np.datetime64(max_date), pd.to_datetime(response_df.date) >= np.datetime64(min_date))

response_df_last_month = response_df.loc[mask, :]

response_df_last_month.sort_values("date")

In [ ]:
response_dict = response.json()
response_dict.keys()

In [ ]:
response_df.shape[0] == response_dict['length']

In [ ]:
response_dict['maxPageLimit']

In [ ]:
response_dict['requestPayload']

Vemos que el response trajo el resultado de la consulta en una única página porque vemos que `response_dict['pagination']["next"]` es nulo

In [ ]:
response_dict['pagination']

In [ ]:
response_dict['pagination']["next"] is None

Alternativa: 

Hacer un ciclo iterando sobre cada fecha de interés. Usar sleep entre consultas sucesivas https://www.programiz.com/python-programming/time/sleep
    

## Ejercicio 3

Repetir el ejercicio 1 usando la bilbioteca que provee Public Health England

Documentación: https://publichealthengland.github.io/coronavirus-dashboard-api-python-sdk/pages/examples/general_use.html#

In [ ]:
AREA_TYPE = "nation"
AREA_NAME = "england"
DATE = str(ayer)

ej3_filters = [
    f"areaType={ AREA_TYPE }",
    f"areaName={ AREA_NAME }",
    f"date={ DATE }"
]

#print(ej3_filters)

# estructura de la respuesta

ej3_structure = {
    "date": "date",
    "name": "areaName",
    "code": "areaCode",
    "dailyCases": "newCasesByPublishDate",
    "cumulativeCases": "cumCasesByPublishDate",
    "dailyDeaths": "newDeaths28DaysByPublishDate",
    "cumulativeDeaths": "cumDeaths28DaysByPublishDate"
}

api = Cov19API(filters = ej3_filters, structure = ej3_structure)

data_df = api.get_dataframe()

data_df

In [ ]:
data_json = api.get_json()
data_json

In [ ]:
data_csv = api.get_csv()
data_csv

## Ejercicio 4

Repetir el ejercicio 2 usando la bilbioteca que provee Public Health England


In [ ]:
AREA_TYPE = "nation"
AREA_NAME = "england"

ej4_filters = [
    f"areaType={ AREA_TYPE }",
    f"areaName={ AREA_NAME }"
]

# estructura de la respuesta

ej4_structure = {
    "date": "date",
    "name": "areaName",
    "code": "areaCode",
    "dailyCases": "newCasesByPublishDate",
    "cumulativeCases": "cumCasesByPublishDate",
    "dailyDeaths": "newDeaths28DaysByPublishDate",
    "cumulativeDeaths": "cumDeaths28DaysByPublishDate"
}

api = Cov19API(filters = ej4_filters, structure = ej4_structure)

data_df = api.get_dataframe()

data_df

Filtramos por las fechas de interés

In [ ]:
max_date = date.today()
min_date = max_date + timedelta(days=-30)

mask = np.logical_and(pd.to_datetime(data_df.date) <= np.datetime64(max_date), pd.to_datetime(data_df.date) >= np.datetime64(min_date))

data_df_last_month = data_df.loc[mask, :]

data_df_last_month.sort_values("date")

## Referencias y Material Adicional
---

https://coronavirus.data.gov.uk/details/developers-guide

https://github.com/publichealthengland/coronavirus-dashboard-api-python-sdk

https://apidocs.data.world/toolkit/api/clients

https://apidocs.data.world/toolkit/rest-api

https://github.com/datadotworld/data.world-py

https://datosgobar.github.io/series-tiempo-ar-api/

https://datosgobar.github.io/series-tiempo-ar-api/python-usage/